# Yolo Nas Classifier
## notebook complete

## Data Loading and Exploration:

### Library import

In [ ]:
from super_gradients import Trainer
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import (
    PPYoloEPostPredictionCallback
)
from super_gradients.training import Trainer, training_hyperparams
from super_gradients.training.processing.processing import ComposeProcessing, NormalizeImage, Resize, StandardizeImage, ImagePermute

from super_gradients.training import models
from super_gradients.common.object_names import Models


In [ ]:
import random
import os
import glob
import numpy as np
import seaborn as sns
import pprint
import torch
from torch import rand, randint
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_recall_curve, average_precision_score, PrecisionRecallDisplay, roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
from PIL import Image
from matplotlib import pyplot as plt

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

### Paths organization

In [ ]:
# Dataset
pwd_notebook = os.path.abspath('') # path notebook
root_path = os.path.dirname(os.path.dirname(pwd_notebook)) #path root project

data_dir = os.path.join(root_path, 'data') #path data

# train path
train_dir = os.path.join(data_dir, 'train')
print(train_dir)
# test path
test_dir = os.path.join(data_dir, 'test')
print(test_dir)
#valid path
valid_dir = os.path.join(data_dir, 'valid')
print(valid_dir)

In [ ]:
# Checkpoints
experiment_dir = os.path.join(root_path, 'checkpoints', 'class_mam')
CHECKPOINT_DIR = os.path.join(experiment_dir, "class_ckpt")
model_name = 'resnet18'
trainer = Trainer(experiment_name=experiment_dir, ckpt_root_dir=CHECKPOINT_DIR)
trainer

## Data processing

### Data transformation

In [ ]:
# Write transform for image
data_transform = Compose([
    # Resize the images to 64x64
    # transforms.Resize(size=(64, 64)),
    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    # Turn the image into a torch.Tensor
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0 
])

### Data preprocessing

In [ ]:
def plot_transformed_images(image_paths, transform, n=3, seed=40):
    """Plots a series of random images from image_paths.

    Will open n image paths from image_paths, transform them
    with transform and plot them side by side.

    Args:
        image_paths (list): List of target image paths. 
        transform (PyTorch Transforms): Transforms to apply to images.
        n (int, optional): Number of images to plot. Defaults to 3.
        seed (int, optional): Random seed for the random generator. Defaults to 42.
    """
    # random.seed(seed)
    random_image_paths = random.sample(image_paths, k=n)
    for image_path in random_image_paths:
        with Image.open(image_path) as f:

            fig, ax = plt.subplots(1, 2)

            # Normalize and display image in "I" mode
            if f.mode == "I":
                f_normalized = (np.array(f) - np.min(f)) / (np.max(f) - np.min(f)) * 255
                ax[0].imshow(f_normalized)
            else:
                ax[0].imshow(f) 
                
            ax[0].set_title(f"Original \nSize: {f.size} \nMode:{f.mode}")    
            ax[0].axis("off")

            # Transform and plot image
            # Note: permute() will change shape of image to suit matplotlib 
            # (PyTorch default is [C, H, W] but Matplotlib is [H, W, C])
            transformed_image = transform(f).permute(1, 2, 0) 
            ax[1].imshow(transformed_image) 
            ax[1].set_title(f"Transformed \nSize: {transformed_image.shape} \nMode:{f.mode}")
            ax[1].axis("off")

            image_class = os.path.basename(os.path.dirname(image_path))

            fig.suptitle(f"Class: {image_class}", fontsize=16)
            plt.show()

In [ ]:
# Set seed
# random.seed(99) # <- try changing this and see what happens
# 1. Get all image paths (* means "any combination")
image_path_list = glob.glob(os.path.join(data_dir, '**', '*.png'), recursive=True)
# 2. Get random image path
random_image_path = random.choice(image_path_list)
# 3. Get image class from path name (the image class is the name of the directory where the image is stored)
image_class = os.path.basename(os.path.dirname(random_image_path))
# 4. Open image
img = Image.open(random_image_path)
# 5. Print metadata
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}") 
print(f"Image width: {img.width}")
print(f"Image mode: {img.mode}")
print(f"Dataset Size: {len(image_path_list)}")
print(image_path_list)
img

In [ ]:
# Show images
plot_transformed_images(image_path_list, 
                        transform=data_transform, 
                        n=3)

In [ ]:
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

valid_data = datasets.ImageFolder(root=valid_dir, 
                                 transform=data_transform,
                                 target_transform=None)

test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform,
                                 target_transform=None)

print(f"Train data:\n{train_data}\nValid data:\n{valid_data}\nTest data:\n{test_data}")

In [ ]:
# Get class names as a list
class_names = train_data.classes
print(class_names)
# Can also get class names as a dict
class_dict = train_data.class_to_idx
print(class_dict)
# Check the lengths
len(train_data), len(valid_data)

In [ ]:
img, label = train_data[5][0], train_data[5][1]
print(f"Image tensor:\n{img}")
print(f"Image shape: {img.shape}")
print(f"Image datatype: {img.dtype}")
print(f"Image label: {label}")
print(f"Label datatype: {type(label)}")

In [ ]:
# Rearrange the order of dimensions
img_permute = img.permute(1, 2, 0)

# Print out different shapes (before and after permute)
print(f"Original shape: {img.shape} -> [color_channels, height, width]")
print(f"Image permute shape: {img_permute.shape} -> [height, width, color_channels]")

# Plot the image
plt.figure(figsize=(10, 7))
plt.imshow(img.permute(1, 2, 0))
plt.axis("off")
plt.title(class_names[label], fontsize=14)

## Training settings

### Training preparation

In [ ]:
train_dataloader = DataLoader(dataset=train_data, 
                              batch_size=16, # how many samples per batch?
                              num_workers=10, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

valid_dataloader = DataLoader(dataset=valid_data, 
                             batch_size=16, 
                             num_workers=10, 
                             shuffle=True) # don't usually need to shuffle testing data

test_dataloader = DataLoader(dataset=test_data, 
                             batch_size=16, 
                             num_workers=20, 
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, valid_dataloader, test_dataloader

In [ ]:
img, label = next(iter(train_dataloader))

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")
print(f"Train data:\n{train_data}\nValid data:\n{valid_data}\nTest data:\n{test_data}")

### Model

In [ ]:
model = models.get(model_name=Models.RESNET18, num_classes=6, pretrained_weights="imagenet")

In [ ]:
print(model.train)

### Training settings

In [ ]:
# you can see more recipes in super_gradients/recipes
training_params =  training_hyperparams.get("training_hyperparams/imagenet_resnet50_train_params")

pprint.pprint("Training parameters")
pprint.pprint(training_params)

In [ ]:
training_params["max_epochs"] = 5
training_params["sg_logger_params"]["launch_tensorboard"] = True

### Training visualization

In [ ]:
log_dir = os.path.join(root_path, 'logs', 'training_logs')

%load_ext tensorboard
%tensorboard --logdir log_dir

### Train

In [ ]:
trainer.train(model=model,
              training_params=training_params,
              train_loader=train_dataloader,
              valid_loader=valid_dataloader)


## Evaluation

### Evaluation and testing

In [ ]:
# Get an image from the test set (replace 'index' with the index of the image you want)
index = 2  # Change this to the desired image index
image, true_label = test_data[index]  # Get the image and its true label

# Perform model inference on the image
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    image = image.unsqueeze(0)  # Add a batch dimension since the model expects batches
    output = model(image.to(device))
    _, predicted_label = torch.max(output, 1)  # Get the predicted label

# Get the name of the predicted class from the index
predicted_class = class_names[predicted_label.item()]
true_class = class_names[true_label]

# Show image along with predicted and true label
plt.imshow(image.squeeze().permute(1, 2, 0))  # Show image (make sure dimensions are appropriate)
plt.title(f"Predicted: {predicted_class}\nTrue: {true_class}")
plt.axis('off')
plt.show()

### Accuracy and Classification report

In [ ]:
all_preds = []
all_labels = []

# Evaluate the model on the test data set
model.eval()
with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate general metrics
accuracy = accuracy_score(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=class_names)
# display = PrecisionRecallDisplay.from_estimator(preds, report, accuracy)

print(f"Accuracy: {accuracy}\n")
print("Classification Report:\n", report)

### Confusion matrix

In [ ]:
# Draw confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='g', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

### Show predicts

In [ ]:
def visualize_predictions(index=None):
    if index is None:
        index = random.randint(0, len(test_data)-1)
    image, true_label = test_data[index]
    model.eval()
    with torch.no_grad():
        image_tensor = image.unsqueeze(0).to(device)
        output = model(image_tensor)
        _, predicted_label = torch.max(output, 1)
        predicted_class = class_names[predicted_label.item()]
        true_class = class_names[true_label]

    plt.imshow(image.permute(1, 2, 0))  # Ensure dimensions are appropriate
    plt.title(f"Predicted: {predicted_class}\nTrue: {true_class}")
    plt.axis('off')
    plt.show()

# View some random images with your predictions
for _ in range(5):
    visualize_predictions()


### show graph Recall-Precision curve

#### Show Recall-Precision Particular

In [ ]:
#Obtain Probabilities of each Class

y_true = []
y_scores = []

model.eval()
with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1).cpu().numpy()

        y_true.extend(labels.cpu().numpy())
        y_scores.extend(probabilities)

y_true = np.array(y_true)
y_scores = np.array(y_scores)


In [ ]:
# Binarize tags in a one-vs-all format
y_true_bin = label_binarize(y_true, classes=np.arange(len(class_names)))

# Calculate precision and recall for each class
for i in range(len(class_names)):
    precision, recall, _ = precision_recall_curve(y_true_bin[:, i], y_scores[:, i])
    
    plt.figure()
    plt.plot(recall, precision, marker='.')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve for {class_names[i]}')
    plt.show()


#### Show Precision-Recall Multiclass

In [ ]:
# Binarize tags
y_true_bin = label_binarize(y_true, classes=np.arange(len(class_names)))
n_classes = len(class_names)

# Calculate average precision and recall for each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_scores[:, i])
    average_precision[i] = average_precision_score(y_true_bin[:, i], y_scores[:, i])

# Colors for different classes
colors = cycle(['blue', 'red', 'green', 'yellow', 'orange', 'purple'])

# Draw Precision-Recall curves and iso-F1 curves
plt.figure(figsize=(7, 8))
f_scores = np.linspace(0.2, 0.8, num=6)
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    # l, = plt.plot(x[y >= 0], y[y >= 0], color="gray", alpha=0.2) # iso-F1 curves
    plt.annotate(f"f1={f_score:0.1f}", xy=(0.9, y[45] + 0.02))

for i, color in zip(range(n_classes), colors):
    plt.plot(recall[i], precision[i], color=color, lw=2,
             label=f'Precision-Recall curve of {class_names[i]} (AP = {average_precision[i]:0.2f})')

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="best")
plt.title("Precision-Recall curve to multi-class")
plt.show()


### Show ROC curve

#### ROC CURVE || FPR vs TNR

In [ ]:
# Binarize tags
y_true_bin = label_binarize(y_true, classes=np.arange(len(class_names)))
n_classes = len(class_names)

# Calculate ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Colors for different classes
colors = cycle(['blue', 'red', 'green', 'yellow', 'orange', 'purple'])

# Draw the ROC curve for each class
plt.figure(figsize=(10, 8))
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'ROC curve of {class_names[i]} (area = {roc_auc[i]:0.2f})')

# plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Draw line for classifier
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (Fall out)')
plt.ylabel('True Positive Rate (Sensivity)')
plt.title('ROC Curve for Multi-Class FPR vs TPR')
plt.legend(loc="lower right")
plt.show()


#### ROC CURVE || Specificity vs Sensibility

In [ ]:
# Binarize tags
y_true_bin = label_binarize(y_true, classes=np.arange(len(class_names)))
n_classes = len(class_names)

# Colors for different classes
colors = cycle(['blue', 'red', 'green', 'yellow', 'orange', 'purple'])

plt.figure(figsize=(10, 8))

# Calculate ROC curve (TNR and TPR) and ROC area (AROC) for each class
for i, color in zip(range(n_classes), colors):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
    tnr = 1 - fpr
    roc_auc = auc(tnr, tpr)

    plt.plot(tnr, tpr, color=color, lw=2, label=f'{class_names[i]} (AROC = {roc_auc:.2f})')

plt.xlabel('True Negatives Rate (TNR - Specificity)')
plt.ylabel('True Positive Rate (TPR - Sensibility)')
plt.title('ROC Curve for Multi-Class Specificity vs Sensibility')
plt.legend(loc="best")
plt.show()
